<a href="https://colab.research.google.com/github/sejeong-park/2020-AI/blob/master/TFT%EB%AA%A8%EB%B0%94%EC%9D%BC_%EC%8A%B9%EB%B0%A9%EB%A5%A0_%EC%8A%B9%ED%8C%A8%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.6.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=8a7fb3fbaa9e39c899ccc0e651b0a55cb87a4438a3a8a870bebabcdadf189039
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 67 Jun 15 08:28 kaggle.json


In [ ]:
! kaggle competitions download -c tft-hyona

  0% 0.00/218k [00:00<?, ?B/s]
100% 218k/218k [00:00<00:00, 70.0MB/s]


In [ ]:
!unzip tft-hyona.zip

Archive:  tft-hyona.zip
  inflating: moral_TFT_test.csv      
  inflating: moral_TFT_train.csv     
  inflating: sample.csv              


In [ ]:
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler # For Normalization

In [ ]:
train_csv = pd.read_csv('moral_TFT_train.csv')
train_D = train_csv.drop('Ranked',axis = 1)
train_L = train_csv.Ranked
test_D = pd.read_csv('moral_TFT_test.csv')

In [ ]:
train_D

,JarvanIV,Sona,Rakan,XinZhao,Neeko,Karma,Soraka,Lulu,Malphite,Yasuo,MasterYi,Jinx,Kayle,MissFortune,AurelionSol,KaiSa,Annie,Shaco,Rumble,Lux,Fizz,Ekko,Gangplank,Ziggs,Graves,Blitzcrank,Lucian,Ezreal,Vi,ChoGath,Poppy,Mordekaiser,Jayce,Ashe,WuKong,Jhin,KhaZix,Fiora,Leona,Irelia,Xayah,Kassadin,Thresh,Ahri,Syndra,Zoe,VelKoz,Shen,Darius,Caitlyn,TwistedFate,DarkStar,Protector,Rebel,Set3_Celestial,Set3_Mystic,StarGuardian,TemplateTrait,Blaster,Mercenary,Set3_Blademaster,Set3_Brawler,Starship,Valkyrie,Cybernetic,Demolitionist,Infiltrator,MechPilot,Set3_Sorcerer,SpacePirate,Chrono,Set3_Void,Sniper,Vanguard,ManaReaver
0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,1.0,3.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,2.0,1.0,0.0,2.0,1.0,3.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,4.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,1.0,2.0,4.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,2.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,2.0,0.0,0.0,2.0,2.0,2.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,3.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,2.0
6696,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0
6697,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,2.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,4.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
6698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,1.0,3.0,0.0,0.0,0.0


In [ ]:
train_L

tensor([0, 0, 0,  ..., 0, 0, 0])

In [ ]:
train_D= torch.FloatTensor(np.array(train_D))
train_L = torch.LongTensor(np.array(train_L))
test_D = torch.FloatTensor(np.array(test_D))

In [ ]:
train_L

tensor([0, 0, 0,  ..., 0, 0, 0])

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset=torch.utils.data.TensorDataset(train_D, train_L),
                                          batch_size=100,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
linear1= torch.nn.Linear(train_D.shape[1], 256, bias = True)
linear2 = torch.nn.Linear(256,256,bias = True)
linear3 = torch.nn.Linear(256,256,bias = True)
linear4 = torch.nn.Linear(256,256,bias = True)
linear5 = torch.nn.Linear(256,2, bias=  True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p = 0.3)

In [ ]:
torch.nn.init.xavier_normal(linear1.weight)
torch.nn.init.xavier_normal(linear2.weight)
torch.nn.init.xavier_normal(linear3.weight)
torch.nn.init.xavier_normal(linear4.weight)
torch.nn.init.xavier_normal(linear5.weight)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  """


Parameter containing:
tensor([[-1.1016e-01,  3.8220e-02, -3.3337e-02,  1.9869e-02,  2.8343e-02,
          2.0104e-01,  1.3599e-01, -8.4806e-02,  5.5926e-02,  8.8898e-03,
         -5.2970e-04,  4.6597e-02,  3.5502e-02, -3.1195e-02, -7.5791e-02,
         -3.7435e-02,  4.7421e-02,  9.3494e-02, -5.8333e-02, -4.8365e-02,
         -1.5863e-01,  1.0921e-01,  2.3679e-01, -4.2331e-03, -1.3422e-02,
         -3.5500e-02,  7.0703e-02, -1.2928e-01, -8.6596e-02,  1.0253e-02,
          2.6316e-02,  1.9953e-02, -1.0991e-02, -1.1355e-02, -9.2926e-02,
          4.1886e-02, -2.3508e-02, -8.7822e-02,  6.8931e-03,  8.4693e-02,
          2.9858e-02, -6.7298e-02, -7.3263e-02, -3.0865e-02,  7.9273e-02,
         -1.0065e-01, -9.0814e-02,  1.5167e-01,  2.4391e-01,  6.7645e-02,
          4.4424e-02,  9.6180e-02, -4.5756e-02, -3.7919e-03, -2.4857e-02,
         -1.7738e-02, -7.5452e-02,  8.1756e-02, -1.1287e-01,  5.2770e-03,
         -5.8502e-03,  2.6415e-03, -6.4181e-02, -1.3450e-01, -4.2162e-02,
          7.3240

In [ ]:
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5)

In [ ]:
loss =torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
import torch.nn.functional as F

total_batch = len(data_loader)

model.train()
for e in range(15):

  avg_cost= 0

  for x, y in data_loader:

    X = x.to(device)
    Y=  y.to(device)

    hypothesis= model(X)
    cost = loss(hypothesis, Y)
   
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch {}'.format(e), 'cost {}'.format(avg_cost))

Epoch 0 cost 0.6916729211807251
Epoch 1 cost 0.6258869171142578
Epoch 2 cost 0.5925130248069763
Epoch 3 cost 0.5614397525787354
Epoch 4 cost 0.5490742921829224
Epoch 5 cost 0.527576744556427
Epoch 6 cost 0.5144951939582825
Epoch 7 cost 0.505511462688446
Epoch 8 cost 0.49723097681999207
Epoch 9 cost 0.49319297075271606
Epoch 10 cost 0.48308154940605164
Epoch 11 cost 0.477673202753067
Epoch 12 cost 0.4728682041168213
Epoch 13 cost 0.46600937843322754
Epoch 14 cost 0.4658046066761017


In [ ]:
test_D = torch.FloatTensor(np.array(test_D))

In [ ]:
test_D

tensor([[2., 3., 3.,  ..., 2., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 2.],
        [0., 0., 0.,  ..., 0., 1., 2.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 2.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 2.]])

In [ ]:
with torch.no_grad():
  model.eval()
  test_D=test_D.to(device)
  prediction=model(test_D)

In [ ]:
prediction[:,[-1]]

tensor([[-0.7346],
        [ 0.6002],
        [ 0.7821],
        ...,
        [ 0.9017],
        [ 0.0718],
        [ 1.2547]])

In [ ]:
result = pd.read_csv('sample.csv')

In [ ]:
result['id'] = list(i for i in range(3300))
result['result'] = torch.argmax(pred, 1).cpu()

In [ ]:

result.to_csv('submission.csv', index = False)

In [ ]:
!kaggle competitions submit -c tft-hyona -f submission.csv -m "17011885"

100% 36.5k/36.5k [00:00<00:00, 153kB/s]
Successfully submitted to TFT 모바일 승방률(승/패)